In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-10-02 14:35:37--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2021-10-02 14:35:38 (6.02 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("AmazonHealthPersonalCareReviews").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Health_Personal_Care_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Health_Personal_Care_v1_00.tsv.gz"), sep="\t", header=True)

# Show DataFrame
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|     650634| R3EQSTM9PWRAL|B0091LBZSU|     578484426|Demograss Capsule...|Health & Personal...|          3|            0|          0|   N|                Y|         Three Stars|Only came with 30...| 2015-08-31|
|         US|   19827510| RBWPRK17XKIXD|B00PWW3LQ6|     456433146|Viva Labs #1 Prem...|Health & Personal

In [5]:
df.count()

5331449

In [6]:
HPC_df = df.dropna()
HPC_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|     650634| R3EQSTM9PWRAL|B0091LBZSU|     578484426|Demograss Capsule...|Health & Personal...|          3|            0|          0|   N|                Y|         Three Stars|Only came with 30...| 2015-08-31|
|         US|   19827510| RBWPRK17XKIXD|B00PWW3LQ6|     456433146|Viva Labs #1 Prem...|Health & Personal

In [7]:
HPC_df.count()

5330701

In [8]:
HPC_df.dropDuplicates().show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    7070765|R1004EU9CH0I12|B007IOTMMS|      39630933|          PostureNOW|Health & Personal...|          2|            0|          0|   N|                Y|           Two Stars|hard for a solo p...| 2015-03-13|
|         US|   35990546|R100N7F2NN1839|B0071FQ0JW|     999091819|910mg Pure Green ...|Health & Personal

In [9]:
HPC_df.dropDuplicates().count()

5330701

In [10]:
HPC_df = HPC_df.dropDuplicates()

In [11]:
HPC_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [12]:
from pyspark.sql.types import IntegerType
HPC_df = HPC_df.withColumn("customer_id",HPC_df["customer_id"].cast(IntegerType()))
HPC_df = HPC_df.withColumn("product_parent",HPC_df["product_parent"].cast(IntegerType()))
HPC_df = HPC_df.withColumn("star_rating",HPC_df["star_rating"].cast(IntegerType()))
HPC_df = HPC_df.withColumn("helpful_votes",HPC_df["helpful_votes"].cast(IntegerType()))
HPC_df = HPC_df.withColumn("total_votes",HPC_df["total_votes"].cast(IntegerType()))

In [13]:
print(HPC_df.select("customer_id").dtypes)
print(HPC_df.select("product_parent").dtypes)
print(HPC_df.select("star_rating").dtypes)
print(HPC_df.select("helpful_votes").dtypes)
print(HPC_df.select("total_votes").dtypes)

[('customer_id', 'int')]
[('product_parent', 'int')]
[('star_rating', 'int')]
[('helpful_votes', 'int')]
[('total_votes', 'int')]


In [14]:
HPC_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



Trnasforming dataset to fit tables in the Schema File

In [15]:
from pyspark.sql.functions import to_date
review_id = HPC_df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1004EU9CH0I12|    7070765|B007IOTMMS|      39630933| 2015-03-13|
|R100N7F2NN1839|   35990546|B0071FQ0JW|     999091819| 2013-11-29|
|R100O14GDT6UI4|   22929146|B004PXM3XA|     391827617| 2015-02-19|
|R100X9ZRM2D3ME|   19807351|B000JE2C9Y|     153422229| 2013-12-04|
|R101GG16T5MKII|   46548996|B00EY0I5F4|     936774947| 2014-09-04|
|R101KRYAQOS3PU|   16846972|B00EIWU2FE|     809738378| 2015-02-19|
|R101V8XFCTOWWQ|    5417052|B00LQDI3PO|     296111002| 2014-09-21|
|R1021S4FXL8EVF|     131307|B0057ZYI9W|     435784364| 2015-08-02|
|R10231DG74M6M2|   32435186|B001DB4MFO|     338391799| 2013-02-17|
|R102KV7SYNWKL3|   20287213|B002M22DBI|     651170219| 2010-04-09|
|R102XTO55YH30C|   38025172|B0001Q6CBY|     373901951| 2014-03-13|
|R1031H02NDL1S5|   19330308|B000146LKS|     387834247| 2013-02

In [17]:
from pyspark import SparkFiles
products = HPC_df.select(["product_id", "product_title"])
products = products.drop_duplicates(["product_id", "product_title"])
products.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00NWYMDJM|Premium Prenatal ...|
|B00B8918MG|Bodylogix Natural...|
|B001D8ZGAM|Aquasentials Easy...|
|B0093AIT7O|Isagenix Ionix Su...|
|B009K6XZ5M|Top Secret Nutrit...|
|B00AYIM9Y8|BulkSupplements P...|
|B000GPCMJG|SE Podiatrist Toe...|
|B00020HQU0|Nature's Way Meli...|
|B00N2BW2PK|Fitbit Charge HR ...|
|B00068JKNO|Swanson Coq10 100...|
|B002HWRY8U|  Emergen-C Lite MSM|
|B0001TQA22|       Mayumi - Anti|
|B00AWJBU2Q|13 mm Anti-slip S...|
|B00115J0TU|Crutch Critters C...|
|B010G18D8S|New My Snoring So...|
|B007TTJQJQ|Poise Pads, Regul...|
|B00HNYES2C|Pure Clay – Premi...|
|B008619IB2|Cepacol Sensation...|
|B0071O8B3G|Lipozene Diet Pil...|
|B00QKC6U38|Detox Cleanse, De...|
+----------+--------------------+
only showing top 20 rows



In [18]:
from pyspark import SparkFiles
# DataFrame
customers = HPC_df.groupby(["customer_id"]).count()
customers = customers.withColumnRenamed("count", "customer_count")
customers.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   11047211|             8|
|   10264170|             1|
|   16551925|             1|
|   32150085|             3|
|   39022482|             1|
|   25004699|             5|
|   17939238|             1|
|   19241294|             1|
|    6935058|             4|
|   23630066|             1|
|   30024215|             1|
|   46333315|             3|
|   39913192|             2|
|   44556866|            10|
|   45654215|             5|
|   10447840|             1|
|   51114551|             3|
|   23809730|             1|
|   22589283|             1|
|   22727082|             6|
+-----------+--------------+
only showing top 20 rows



In [19]:
from pyspark import SparkFiles
vine_table = HPC_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_table.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1004EU9CH0I12|          2|            0|          0|   N|
|R100N7F2NN1839|          3|            0|          0|   N|
|R100O14GDT6UI4|          2|            0|          0|   N|
|R100X9ZRM2D3ME|          2|            1|          1|   N|
|R101GG16T5MKII|          5|            9|          9|   N|
|R101KRYAQOS3PU|          5|            1|          1|   N|
|R101V8XFCTOWWQ|          1|            0|          1|   N|
|R1021S4FXL8EVF|          4|            0|          0|   N|
|R10231DG74M6M2|          5|            0|          0|   N|
|R102KV7SYNWKL3|          2|            0|          0|   N|
|R102XTO55YH30C|          5|            3|          3|   N|
|R1031H02NDL1S5|          4|            0|          0|   N|
|R1032ILCZQROOE|          5|            0|          0|   N|
|R1034S9GX3609B|          1|          17

In [22]:
mode = 'append'
jdbc_url = "jdbc:postgresql://mypostgresdb.ceomfsf1omq9.us-east-2.rds.amazonaws.com:5432/my_data_class_db_HPC"
config = {
    'user':'postgres',
    'password':'********',
    'driver':"org.postgresql.Driver"
}

In [21]:
review_id.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

Py4JJavaError: ignored

In [ ]:
products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)